<a href="https://colab.research.google.com/github/Poulette2002/Video-Search/blob/main/Cluster_AnalysisV5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U clustimage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 6.9 MB/s 
     |████████████████████████████████| 812 kB 41.8 MB/s 
     |████████████████████████████████| 1.1 MB 46.5 MB/s 
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=4e7ece0a29d7853958fe620d171cffe8734b6c5e1de7d2550869152ae217a207
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e297533ddb09407769762985d134135064c6831e29a914e
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=42db880eb75d8ee87184090eb594fa2394fe9dd52ee7ca6908c7803d67641102
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=d139b0ca4674dc9095b7f1346003eb6fbb93f55262637fb4e57916d0f3e02010
  Stored in directory: /root/.cache/

In [2]:
import cv2
import numpy as np
import time
from google.colab.patches import cv2_imshow
from google.colab import files
import matplotlib.pyplot as plt
from clustimage import Clustimage
import os 
from scipy import cluster
import shutil
from datetime import timedelta
import numpy as np

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import clustimage
print(clustimage.__version__)

1.5.5


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
#https://drive.google.com/drive/folders/1c1iJHy6HnH_ZAuRcK9i9nqs0TFXYd4rt?usp=sharing

Mounted at /content/gdrive


In [4]:
# i.e if video of duration 30 seconds, saves 10 frame per second = 300 frames saved in total
SAVING_FRAMES_PER_SECOND = 10

In [5]:
def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05) 
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")


def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

In [9]:
def split_video(video_file, n, f): #n is the number of the video, f is the shotnumber of the video n     
    if n<10:
      filename1 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_0000{n}"
      filename2 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_0000{n}/video_{f}"
      # make a folder by the name of the video file
      if not os.path.isdir(filename1):
        os.mkdir(filename1)
      if not os.path.isdir(filename2):
        os.mkdir(filename2)
    elif  n>=10 and n<100:
      filename1 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_000{n}"
      filename2 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_000{n}/video_{f}"
      # make a folder by the name of the video file
      if not os.path.isdir(filename1):
        os.mkdir(filename1)
      if not os.path.isdir(filename2):
        os.mkdir(filename2)
    else:
      filename1 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00{n}"
      filename2 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00{n}/video_{f}"
      # make a folder by the name of the video file
      if not os.path.isdir(filename1):
        os.mkdir(filename1)
      if not os.path.isdir(filename2):
        os.mkdir(filename2)
    # read the video file    
    cap = cv2.VideoCapture(video_file)
    # get the FPS of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # get the list of duration spots to save
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    # start the loop
    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break
        # get the duration by dividing the frame count by the FPS
        frame_duration = count / fps
        try:
            # get the earliest duration to save
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # the list is empty, all duration frames were saved
            break
        if frame_duration >= closest_duration:
            # if closest duration is less than or equals the frame duration, 
            # then save the frame
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            cv2.imwrite(os.path.join(filename2, f"frame{frame_duration_formatted}.jpg"), frame) 
            # drop the duration spot from the list, since this duration spot is already saved
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # increment the frame count
        count += 1

In [ ]:
for n in range(1,200):
    if n<10:
        for f in range(0,200):
          video_path = (f"/content/gdrive/MyDrive/Video_Splits/video_split_0000{n}/"+f"video_{f}"+ ".avi")
          try: 
            print(video_path)
            split_video(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    elif n<100:
        for i in range(0,200):
          video_path = (f"/content/gdrive/MyDrive/Video_Splits/video_split_000{n}/"+f"video_{f}"+ ".avi")
          try: 
            print(video_path)
            split_video(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    else:
        for i in range(0,200):
          video_path = (f"/content/gdrive/MyDrive/Video_Splits/video_split_00{n}/"+f"video_{f}"+ ".avi")
          try: 
            print(video_path)
            split_video(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    

In [10]:
for n in range(9,10):
    if n<10:
        for f in range(0,200):
          video_path = (f"/content/gdrive/MyDrive/Video_Splits/video_split_0000{n}/"+f"video_{f}"+ ".avi")
          try: 
            print(video_path)
            split_video(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    elif n>=10 and n<100:
        for i in range(0,200):
          video_path = (f"/content/gdrive/MyDrive/Video_Splits/video_split_000{n}/"+f"video_{f}"+ ".avi")
          try: 
            print(video_path)
            split_video(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    else:
        for i in range(0,200):
          video_path = (f"/content/gdrive/MyDrive/Video_Splits/video_split_00{n}/"+f"video_{f}"+ ".avi")
          try: 
            print(video_path)
            split_video(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    

/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_0.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_1.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_2.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_3.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_4.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_5.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_6.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_7.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_8.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_9.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_10.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_11.avi
/content/gdrive/MyDrive/Video_Splits/video_split_00009/video_12.avi
video_12 doesn't excist


In [11]:
def cluster_analysis(path, n, f): #path is the path to the folder with the input images, n is the number of the video and f is the number of shot in the video n  
  if n<10:
    filename1 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_0000{n}"
    filename2 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_0000{n}/video_{f}"
    if not os.path.isdir(filename1):
      os.mkdir(filename1)
    if not os.path.isdir(filename2):
      os.mkdir(filename2)
  
    cl = Clustimage(method='pca')
    results = cl.fit_transform(path , min_clust=1, max_clust=15)
    cl.cluster(cluster='agglomerative', evaluate='dbindex', metric='euclidean', linkage='ward', min_clust=1, max_clust=15, cluster_space='low')
    unique_images = {}
    unique_images = cl.unique()
    
    original = unique_images["pathnames"]
    #target ="/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/Testvideo"
    for each in original:
      shutil.copy(each,filename2)

  elif n<100:
    filename1 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_000{n}"
    filename2 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_000{n}/video_{f}"
    if not os.path.isdir(filename1):
      os.mkdir(filename1)
    if not os.path.isdir(filename2):
      os.mkdir(filename2)
    
    cl = Clustimage(method='pca')
    results = cl.fit_transform(path , min_clust=1, max_clust=15)
    cl.cluster(cluster='agglomerative', evaluate='dbindex', metric='euclidean', linkage='ward', min_clust=1, max_clust=15, cluster_space='low')
    unique_images = {}
    unique_images = cl.unique()
    
    original = unique_images["pathnames"]
    #target ="/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/Testvideo"
    for each in original:
      shutil.copy(each,filename2)
  else:
    filename1 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_00{n}"
    filename2 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_00{n}/video_{f}"
    if not os.path.isdir(filename1):
      os.mkdir(filename1)
    if not os.path.isdir(filename2):
      os.mkdir(filename2)
    
    cl = Clustimage(method='pca')
    results = cl.fit_transform(path , min_clust=1, max_clust=15)
    cl.cluster(cluster='agglomerative', evaluate='dbindex', metric='euclidean', linkage='ward', min_clust=1, max_clust=15, cluster_space='low')
    unique_images = {}
    unique_images = cl.unique()
    
    original = unique_images["pathnames"]
    #target ="/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/Testvideo"
    for each in original:
      shutil.copy(each,filename2)

  # cl = Clustimage(method='pca')
  # results = cl.fit_transform(path , min_clust=1, max_clust=15)
  # cl.cluster(cluster='agglomerative', evaluate='dbindex', metric='euclidean', linkage='ward', min_clust=1, max_clust=15, cluster_space='low')
  # unique_images = {}
  # unique_images = cl.unique()
  
  # original = unique_images["pathnames"]
  # #target ="/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/Testvideo"
  # for each in original:
  #   shutil.copy(each,target)


In [14]:
#path = '/content/gdrive/MyDrive/Keyframes_Videos_Part_1/key_frame_video_2'
#target = "/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/Testvideo2"
#cluster_analysis(path, target)

for n in range(1,10):
    if n<10:
        for f in range(0,200):
          video_path = (f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_0000{n}/"+f"video_{f}")
          try: 
            print(video_path)
            cluster_analysis(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    elif n<100:
        for i in range(0,200):
          video_path = (f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_000{n}/"+f"video_{f}")
          try: 
            print(video_path)
            cluster_analysis(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    else:
        for i in range(0,200):
          video_path = (f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00{n}/"+f"video_{f}")
          try: 
            print(video_path)
            cluster_analysis(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break

[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> Cleaning previous fitted model results
[clustimage] >INFO> Extracting images from: [/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00001/video_0]
[clustimage] >INFO> [48] files are collected recursively from path: [/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00001/video_0]
[clustimage] >INFO> [48] images are extracted.
[clustimage] >INFO> Reading and checking images.
[clustimage] >INFO> Reading and checking images.


/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00001/video_0


100%|██████████| 48/48 [00:02<00:00, 19.57it/s]
[clustimage] >INFO> Extracting features using method: [pca]


[pca] >Column labels are auto-completed.
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [49152] columns of the input data.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Number of components is [22] that covers the [95.00%] explained variance.
[pca] >The PCA reduction is performed on the [49152] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.


[clustimage] >INFO> Extracted features using [pca]: samples=48, features=22
[clustimage] >INFO> Compute [tsne] embedding
[clustimage] >INFO> Cluster evaluation using the [high] feature space of the [pca] features.


...

[clusteval] >Compute dendrogram threshold.
[clusteval] >Optimal number clusters detected: [13].
[clusteval] >Fin.


[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> Cleaning previous fitted model results
[clustimage] >INFO> Extracting images from: [/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00008/video_104]
[clustimage] >INFO> [0] files are collected recursively from path: [/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00008/video_104]
[clustimage] >INFO> [0] images are extracted.
[clustimage] >INFO> Reading and checking images.
[clustimage] >INFO> Extracting features using method: [pca]
[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> Cleaning previous fitted model results
[clustimage] >INFO> Reading and checking images.
[clustimage] >INFO> Reading and checking images.


/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00008/video_104
video_104 doesn't excist
/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00009/video_0


0it [00:00, ?it/s]

video_0 doesn't excist
